# **[HiFiPLN ](https://utau.pl/hifipln/)**
Multispeaker Community Vocoder Model for DiffSinger


___
Last update: 02/04/2024


#### **Notebook made by [me](https://github.com/Airi-Lin)**
#### **Inspired by the [Diffsinger_colab_notebook](https://github.com/MLo7Ghinsan/DiffSinger_colab_notebook_MLo7)**

#### **Check the link on HiFiPLN to see the awesome work made! Without them, this wouldnt be possible**
Version: **1.0.0**

___

# **Setup**

In [ ]:
#@markdown **Make sure that your colab session is connected to a GPU.**
!nvidia-smi

In [ ]:
#@markdown **Download repo and install requirements**
from IPython.display import Audio, display, HTML, clear_output
%cd /content/
!git clone https://github.com/Scarfmonster/HiFiPLN/
clear_output()
%cd /content/HiFiPLN
!pip install -r requirements.txt
clear_output()


!wget "https://dl.utau.pl/dl.php?name=checkpoint&file=hifipln&version=1.0&type=7z" -O "hifipln_1.0.7z"
!mkdir /content/HiFiPLN/ckpt
!7za -bso0 -y x "hifipln_1.0.7z" -o/content/HiFiPLN/ckpt
!rm "/content/HiFiPLN/hifipln_1.0.7z"
!wget -O kururin.mp3 https://www.myinstants.com/media/sounds/honkai-herta-hudurin-kuru-kuru.mp3
clear_output()


display(Audio(filename='/content/HiFiPLN/kururin.mp3', autoplay=True, rate=44100))


display(HTML(
      '''
      <marquee ><b>Installation successfully!!</b></marquee>
      <div style = "display: flex; justify-content: left; align-items: left;">
        <img src="https://media.tenor.com/7lHdnabfyTQAAAAi/herta-kurukuru.gif", width=249, height=206>
      </div>
      '''))


In [ ]:
#@markdown **OPTIONAL**
#@markdown Mount Google Drive to this notebook if you have your dataset there
from google.colab import drive
drive.mount("/content/drive")

# **Dataset Preparation**

In [ ]:
#@markdown val_ratio is the percentage of the dataset to be used as validation, going to 0 to 1, being 1 a 100%
dataset_path = "" # @param {type:"string"}

import os
import random
!mkdir /content/HiFiPLN/dataset/valid/

!python dataset-utils/split.py --length 1 -sr 44100 -o "dataset/train" {dataset_path}


# Set the paths of the train and validation folders
train_path = "/content/HiFiPLN/dataset/train"
val_path = "/content/HiFiPLN/dataset/valid"

# Set the fraction of files to move to the validation folder
val_ratio = 0.15 # @param {type:"slider", min:0.05, max:1.0, step:0.05}

# List all the wav files in the train folder
wav_files = [f for f in os.listdir(train_path) if f.endswith(".wav")]

# Shuffle the files
random.shuffle(wav_files)

# Calculate the number of files to move
val_size = int(len(wav_files) * val_ratio)

# Move the files to the validation folder
for i in range(val_size):
  file = wav_files[i]
  src = os.path.join(train_path, file)
  dst = os.path.join(val_path, file)
  os.rename(src, dst)

# Print the number of files in each folder
print(f"Train folder: {len(os.listdir(train_path))} files")
print(f"Validation folder: {len(os.listdir(val_path))} files")


!python preproc.py --path dataset --config "configs/hifipln.yaml"

# **Fine-Tuning**

In [ ]:

#@markdown *Fine-Tuning only available due to possible Colab limits*


validation_steps = 500 # @param {type:"slider", min:500, max:10000, step:500}
learning_rate = 0.00001 #@param {type: "number"}
batch_size = 8 # @param {type:"slider", min:1, max:20, step:1}

from omegaconf import OmegaConf
config = OmegaConf.load("configs/hifipln-finetune.yaml")
config.val_check = validation_steps
config.lr = learning_rate
config.dataloader.train.batch_size = batch_size
config.finetune = "ckpt/hifipln_1.0.ckpt"
config.dataloader.train.num_workers=2
config.dataloader.valid.num_workers=2
OmegaConf.save(config, "configs/config.yaml")


%reload_ext tensorboard
%tensorboard --logdir /content/HiFiPLN/logs/HiFiPLN

!python train.py --config "configs/config.yaml"



# **Export for OpenUTAU**

In [ ]:
Checkpoint = "" # @param {type:"string"}
#@markdown Output should be found on out/hifipln

!python export.py --config configs/hifipln.yaml --output out/hifipln --model {Checkpoint}